### Instructions:
* You need to code in this jupyter notebook only.
* Download this notebokk and import in your jupyter lab.
* You need to write a partial code for step 0 to step 8 mentioned with prefix ##
* Fill the blanks where it is instructed in comments. 
* Leave other codes, structure as it is.
* Follow all the instructions commented in a cells.
* Upload this jupyter notebook after completion with your partial code.
* Also upload the resulting image showing all the selected points and boundary line between them after LDA analysis.
* Duetime: 1:30 PM 

In [1]:
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
matplotlib.use('TkAgg')


In [2]:
Number_of_points = 20  
img = cv2.imread('Indian_Flag.jpg')  # Replace 'your_image_path.jpg' with the actual image path
def select_points(img, title):
    fig, ax = plt.subplots()
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img_rgb)
    plt.title(title)
    fig.canvas.manager.set_window_title('Select Points')
    cursor = matplotlib.widgets.Cursor(ax, useblit=True, color='red', linewidth=1)
    plt.show(block=False)  
    k = 0
    points = []  
    while k < Number_of_points:
        xy = plt.ginput(1, timeout=0)  
        if len(xy) > 0:
            col, row = map(int, xy[0])  
            rgb_value = img[row, col]
            print(f"RGB at ({col}, {row}): {rgb_value}")
            k += 1
            points.append([row, col, img[row, col]]) 
            plt.scatter(col, row, c='black', marker='o', s=10)
            plt.draw()
    plt.close()  
    return points  

In [3]:
pts_saffron = select_points(img, "Select Points from Saffron Strip")
pts_white = select_points(img, "Select Points from White Strip")
pts_green = select_points(img, "Select Points from Green Strip")

RGB at (538, 276): [ 16  88 249]
RGB at (547, 291): [ 12  83 240]
RGB at (577, 291): [ 12  82 243]
RGB at (556, 278): [ 24  94 248]
RGB at (579, 278): [ 20  85 247]
RGB at (559, 293): [ 19  90 248]
RGB at (600, 287): [ 10  81 245]
RGB at (589, 293): [ 17  88 246]
RGB at (591, 273): [ 25  89 251]
RGB at (613, 273): [ 20  84 246]
RGB at (609, 287): [  4  80 246]
RGB at (618, 287): [  9  83 249]
RGB at (628, 282): [ 10  80 247]
RGB at (630, 269): [ 14  82 243]
RGB at (643, 269): [ 14  79 248]
RGB at (641, 278): [  4  75 242]
RGB at (630, 292): [ 12  78 250]
RGB at (645, 290): [ 12  78 243]
RGB at (656, 285): [  1  72 236]
RGB at (656, 267): [ 19  78 241]
RGB at (532, 311): [215 223 223]
RGB at (542, 312): [219 221 221]
RGB at (553, 312): [232 218 229]
RGB at (564, 312): [244 224 236]
RGB at (625, 312): [223 226 224]
RGB at (639, 312): [231 219 219]
RGB at (651, 307): [233 216 219]
RGB at (654, 315): [241 221 220]
RGB at (654, 322): [236 218 219]
RGB at (634, 321): [234 223 225]
RGB at (53

In [6]:
def rgb_to_lab(rgb):
    return cv2.cvtColor(np.uint8([[rgb]]), cv2.COLOR_RGB2Lab)[0][0]

saffron_lab = np.array([rgb_to_lab(rgb) for _, _, rgb in pts_saffron])
white_lab = np.array([rgb_to_lab(rgb) for _, _, rgb in pts_white])
green_lab = np.array([rgb_to_lab(rgb) for _, _, rgb in pts_green])

asaffron = saffron_lab[:, 1] 
bsaffron = saffron_lab[:, 2]  

awhite = white_lab[:, 1]     
bwhite = white_lab[:, 2]      

agreen = green_lab[:, 1]      
bgreen = green_lab[:, 2]    

a_features = np.hstack((asaffron, awhite, agreen))
b_features = np.hstack((bsaffron, bwhite, bgreen))

In [7]:
class_mapping = {'Saffron': 0, 'White': 1, 'Green': 2}
y = np.array([class_mapping[label] for label in ['Saffron'] * Number_of_points + ['White'] * Number_of_points + ['Green'] * Number_of_points])
plt.figure()
plt.scatter(a_features[:Number_of_points], b_features[:Number_of_points], c='b', marker='o', s=50, label='Saffron')
plt.scatter(a_features[Number_of_points:2*Number_of_points], b_features[Number_of_points:2*Number_of_points], c='g', marker='^', s=50, label='White')
plt.scatter(a_features[2*Number_of_points:], b_features[2*Number_of_points:], c='r', marker='*', s=50, label='Green')
plt.legend(['Saffron', 'White', 'Green'], loc='best')
plt.xlabel('color dimension ranging from green (-a*) to red (+a*)')  ## Provide x label
plt.ylabel('color dimension ranging from blue (-b*) to yellow (+b*)')  ## Provide y label
plt.title('LDA Analysi')  
plt.grid()
plt.show()

In [12]:
lda = LDA()
lda.fit(np.column_stack((a_features, b_features)), y)
plt.figure()

plt.scatter(a_features[:Number_of_points], b_features[:Number_of_points], c='b', marker='o', s=50, label='Saffron')
plt.scatter(a_features[Number_of_points:2*Number_of_points], b_features[Number_of_points:2*Number_of_points], c='g', marker='^', s=50, label='White')
plt.scatter(a_features[2*Number_of_points:], b_features[2*Number_of_points:], c='r', marker='*', s=50, label='Green')

plt.xlabel('color dimension ranging from green (-a*) to red (+a*)')  
plt.ylabel('color dimension ranging from blue (-b*) to yellow (+b*)')  
plt.title('LDA boundaries')

ax = plt.gca()
xlim = ax.get_xlim()
ylim = ax.get_ylim()

xx, yy = np.meshgrid(np.linspace(xlim[0], xlim[1], 100), np.linspace(ylim[0], ylim[1], 100))
Z = lda.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

plt.contour(xx, yy, Z, colors='k', linewidths=2, linestyles='solid')
plt.legend(loc='best')
plt.grid()
plt.show()